In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

urls = {
    "2025/26": [
        "https://resultater.volleyball.dk/tms/Turneringer-og-resultater/Pulje-Komplet-Kampprogram.aspx?PuljeId=3752",  # league
        "https://resultater.volleyball.dk/tms/Turneringer-og-resultater/Pulje-Kampprogram.aspx?PuljeId=3998"           # cup
    ],
    "2024/25": [
        "https://resultater.volleyball.dk/tms/Turneringer-og-resultater/Pulje-Komplet-Kampprogram.aspx?PuljeId=3925"
    ]
}

team = "Frederiksberg Volley"
all_matches = []

for season, season_urls in urls.items():
    for url in season_urls:
        resp = requests.get(url)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        table = soup.find("table", {"class": "srDefault srProgramNormal"})
        if not table:
            continue

        for row in table.find_all("tr"):
            cols = row.find_all("td")
            if len(cols) < 6:
                continue

            kampnr = cols[0].get_text(strip=True)
            date_time = cols[1].get_text(" ", strip=True)
            hjemme = cols[2].get_text(strip=True)
            ude = cols[3].get_text(strip=True)
            venue = cols[4].get_text(" ", strip=True)
            result = cols[5].get_text(strip=True)

            if team in hjemme or team in ude:
                match = {
                    "season": season,
                    "kampnr": kampnr,
                    "dato_tid": date_time,
                    "hjemme": hjemme,
                    "ude": ude,
                    "spillested": venue,
                    "resultat": result,
                }
                all_matches.append(match)

df = pd.DataFrame(all_matches)

# --- Add won/lost column ---
def team_result(row):
    try:
        home_sets, away_sets = [int(x) for x in row["resultat"].replace("\xa0", " ").split("-")]
    except Exception:
        return None

    if team == row["hjemme"] and home_sets > away_sets:
        return "won"
    elif team == row["ude"] and away_sets > home_sets:
        return "won"
    else:
        return "lost"

df["result_for_frb"] = df.apply(team_result, axis=1)

df["date"] = df["dato_tid"].str.split(" kl.").str[0]
df["date"] = pd.to_datetime(df["date"], format="%d-%m-%y", errors="coerce").dt.strftime("%Y-%m-%d")

team_abbr = {
    "Frederiksberg Volley": "frb",
    "Team Køge": "tkv",
    "Amager Volley.2": "avk",
    "Lyngby-Gladsaxe VK": "lgv",
    "Hvidovre VK.2": "hv2",
    "Gentofte Volley.2": "gv2",
    "Farum-Holte": "fah"
}

df["match_name"] = df.apply(
    lambda row: f"{team_abbr.get(row['hjemme'], row['hjemme'][:3].lower())}-"
                f"{team_abbr.get(row['ude'], row['ude'][:3].lower())}",
    axis=1
)

frb_name = "Frederiksberg Volley"
df["opponent"] = df.apply(
    lambda row: row["ude"] if row["hjemme"] == frb_name else row["hjemme"],
    axis=1
)
df["played"] = df["resultat"].str.strip().apply(lambda x: "yes" if x != "" else "no")

def assign_season(date):
    year = date.year
    month = date.month
    if month >= 9:
        return f"{year}/{year + 1}"
    else:
        return f"{year - 1}/{year}"

df["date"] = pd.to_datetime(df["date"], errors='coerce')
df["season"] = df["date"].apply(assign_season)

df["date_time_obj"] = pd.to_datetime(df["dato_tid"].str.replace(" kl.", ""), format="%d-%m-%y %H:%M", errors="coerce")
df = df.sort_values(by="date_time_obj", ascending=True).reset_index(drop=True)
df = df.drop(columns=["date_time_obj"])


df.to_csv("match-results-dvbf.csv", index=False)
df


season  kampnr            dato_tid                hjemme  \
0   2024/2025  141063  29-09-24 kl. 10:00     Gentofte Volley.2   
1   2024/2025  141067  06-10-24 kl. 18:00  Frederiksberg Volley   
2   2024/2025  141074  13-10-24 kl. 14:00  Frederiksberg Volley   
3   2024/2025  141078  27-10-24 kl. 11:30                   VLI   
4   2024/2025  141089  02-11-24 kl. 18:00  Frederiksberg Volley   
5   2024/2025  141094  16-11-24 kl. 14:30            DTU Volley   
6   2024/2025  141099  23-11-24 kl. 14:00  Frederiksberg Volley   
7   2024/2025  141104  01-12-24 kl. 14:00            Grøndal EV   
8   2024/2025  141108  08-12-24 kl. 18:00  Frederiksberg Volley   
9   2024/2025  141119  12-01-25 kl. 12:00       Amager Volley.2   
10  2024/2025  141123  18-01-25 kl. 18:00  Frederiksberg Volley   
11  2024/2025  141139  25-01-25 kl. 16:00  Frederiksberg Volley   
12  2024/2025  141084  22-02-25 kl. 16:00  Frederiksberg Volley   
13  2024/2025  141112  08-03-25 kl. 16:00                 KV 61   
14  2024/2025  141134  14-03-25 kl. 19:00             Team Køge   
15  2024/2025  141149  23-03-25 kl. 16:00  Frederiksberg Volley   
16  2024/2025  141144  29-03-25 kl. 17:30         Hvidovre VK.2   
17  2024/2025  141129  06-04-25 kl. 15:00           Farum-Holte   
18  2025/2026  144591  27-09-25 kl. 17:00       Amager Volley.2   
19  2025/2026  144595  04-10-25 kl. 18:00         Hvidovre VK.2   
20  2025/2026  147434  12-10-25 kl. 13:00         Hvidovre VK.2   
21  2025/2026  144608  26-10-25 kl. 16:00            Enghave VK   
22  2025/2026  144610  08-11-25 kl. 14:00  Frederiksberg Volley   
23  2025/2026  144619  23-11-25 kl. 12:00            Grøndal EV   
24  2025/2026  144620  30-11-25 kl. 09:30                   VLI   
25  2025/2026  144604  07-12-25 kl. 13:30  Frederiksberg Volley   
26  2025/2026  144625  14-12-25 kl. 15:30  Frederiksberg Volley   
27  2025/2026  144631  11-01-26 kl. 16:30     Gentofte Volley.2   
28  2025/2026  144635  17-01-26 kl. 16:30            DTU Volley   
29  2025/2026  144643  25-01-26 kl. 14:00  Frederiksberg Volley   
30  2025/2026  144647  07-02-26 kl. 14:00  Frederiksberg Volley   
31  2025/2026  144651  22-02-26 kl. 16:30           Farum-Holte   
32  2025/2026  144657  01-03-26 kl. 15:30  Frederiksberg Volley   
33  2025/2026  144662  15-03-26 kl. 11:30  Frederiksberg Volley   
34  2025/2026  144667  28-03-26 kl. 13:30             Team Køge   
35  2025/2026  144670  12-04-26 kl. 09:30  Frederiksberg Volley   
36  2025/2026  144675  19-04-26 kl. 15:30  Frederiksberg Volley   

                     ude                 spillested resultat result_for_frb  \
0   Frederiksberg Volley          Kildeskovshal 1 1    3 - 2           lost   
1                  KV 61          Bülowsvejhallen 1    3 - 0            won   
2        Amager Volley.2              Kedelhallen 1    0 - 3           lost   
3   Frederiksberg Volley              Kedelhallen 1    3 - 0           lost   
4              Team Køge          Bülowsvejhallen 1    3 - 0            won   
5   Frederiksberg Volley         Engelsborghallen 1    2 - 3            won   
6          Hvidovre VK.2          Bülowsvejhallen 1    2 - 3           lost   
7   Frederiksberg Volley  Hillerødgades Sportssal 1    0 - 3            won   
8      Gentofte Volley.2          Bülowsvejhallen 1    3 - 1            won   
9   Frederiksberg Volley              Sundbyhal 1 1    3 - 2           lost   
10                   VLI          Bülowsvejhallen 1    3 - 2            won   
11            DTU Volley          Bülowsvejhallen 1    3 - 1            won   
12           Farum-Holte          Bülowsvejhallen 1    0 - 3           lost   
13  Frederiksberg Volley              Idrætshuset 1    0 - 3            won   
14  Frederiksberg Volley          Spar Nord Arena 1    2 - 3            won   
15            Grøndal EV          Bülowsvejhallen 1    3 - 1            won   
16  Frederiksberg Volley   Frihedens Idrætscenter 1    0 - 3            won   
17  Frederiksberg Volley              